In [12]:
#import needed libraries
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Embedding, Input, Flatten
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout, AveragePooling1D, AveragePooling2D
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.cross_validation import train_test_split


In [45]:
#read the input dataset
df = pd.read_csv("./train.csv")
#split the data into train and test sets
train, test = train_test_split(df, test_size=0.2)

#Replace all blank comments with text in training set
#extract training comments 
comments_train = train["comment_text"].fillna("cbarcelon").values
#extract the toxciity ratings
classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
train_ratings = train[classes].values
test_ratings = test[classes].values
#extract test comments
comments_test = test["comment_text"].fillna("cbarcelon").values

#tokenizer the text
#vectorize text
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(list(comments_train))
tokenized_comments_train = tokenizer.texts_to_sequences(comments_train)
tokenized_comments_test = tokenizer.texts_to_sequences(comments_test)
#pad the text so each comment is uniform in length
X_train = sequence.pad_sequences(tokenized_comments_train, maxlen=100, truncating='post')
X_test = sequence.pad_sequences(tokenized_comments_test, maxlen=100,  truncating='post')

In [52]:
#define LSTM sequential model
lstm = Sequential()
lstm.add(Embedding(20000, output_dim=128, input_length=100))
lstm.add(Bidirectional(LSTM(75, return_sequences=True), merge_mode='ave'))
lstm.add(AveragePooling1D())
lstm.add(Bidirectional(LSTM(50, return_sequences=True), merge_mode='ave'))
lstm.add(AveragePooling1D())
lstm.add(Bidirectional(LSTM(25, return_sequences=True), merge_mode='ave'))
lstm.add(AveragePooling1D())
lstm.add(Flatten())
lstm.add(Dropout(.5))
lstm.add(Dense(200, activation='relu'))
lstm.add(Dropout(.5))
lstm.add(Dense(6, activation = "sigmoid"))

lstm.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_18 (Embedding)     (None, 100, 128)          2560000   
_________________________________________________________________
bidirectional_52 (Bidirectio (None, 100, 75)           122400    
_________________________________________________________________
average_pooling1d_50 (Averag (None, 50, 75)            0         
_________________________________________________________________
bidirectional_53 (Bidirectio (None, 50, 50)            50400     
_________________________________________________________________
average_pooling1d_51 (Averag (None, 25, 50)            0         
_________________________________________________________________
bidirectional_54 (Bidirectio (None, 25, 25)            15200     
_________________________________________________________________
average_pooling1d_52 (Averag (None, 12, 25)            0         
__________

In [47]:
#compile the model
lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#create checkpoint file
file_path = "weights_base.best.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

#early stop checkpoint
early = EarlyStopping(monitor='val_loss', mode='min', patience=20)
callbacks_list = [checkpoint, early] 

In [48]:
#train/fit the model
lstm.fit(X_train, train_ratings, batch_size=750, epochs=10, validation_split=0.1, callbacks=callbacks_list)

Train on 69012 samples, validate on 7668 samples
Epoch 1/200
69012/69012 [==============================] - 65s - loss: 0.1985 - acc: 0.9596 - val_loss: 0.1176 - val_acc: 0.9652
Epoch 2/200
69012/69012 [==============================] - 62s - loss: 0.0961 - acc: 0.9679 - val_loss: 0.0703 - val_acc: 0.9761
Epoch 3/200
69012/69012 [==============================] - 62s - loss: 0.0639 - acc: 0.9781 - val_loss: 0.0581 - val_acc: 0.9807
Epoch 4/200
69012/69012 [==============================] - 62s - loss: 0.0532 - acc: 0.9811 - val_loss: 0.0606 - val_acc: 0.9786
Epoch 5/200
69012/69012 [==============================] - 62s - loss: 0.0466 - acc: 0.9829 - val_loss: 0.0574 - val_acc: 0.9792
Epoch 6/200
69012/69012 [==============================] - 62s - loss: 0.0421 - acc: 0.9842 - val_loss: 0.0588 - val_acc: 0.9790
Epoch 7/200
69012/69012 [==============================] - 62s - loss: 0.0386 - acc: 0.9855 - val_loss: 0.0593 - val_acc: 0.9799
Epoch 8/200
69012/69012 [=======================

In [49]:
#load the best weights
lstm.load_weights(file_path)

#make predictions on test set
pred = lstm.predict(X_test)

In [50]:
#log loss score function
from sklearn.metrics import log_loss
def calc_loss(y_true, y_pred):
    return np.mean([log_loss(y_true[:, i], y_pred[:, i]) 
                    for i in range(y_true.shape[1])])

In [51]:
score = calc_loss(test_ratings, pred)
print(score)

0.0621659652706
